# Lexicographic optimization

[Lexicographic optimization](https://en.wikipedia.org/wiki/Lexicographic_optimization) is a kind of multiobjective optimization. The basic idea being that the objectives can be ranked according to their importance, and that a small improvement of the most important objective is always seen better as any improvement of the second one.

In discrete-optimization, a sequential algorithm is implemented for lexicographic optimization and available with the meta-solver `LexicoSolver`.
It is wrapping another solver, and performs sequentially mono-objective optimizations on each objective, starting from the most important, and adding a new constraint on the last objective at the end of each step, to avoid degrading it.

Pseudo-code would be:

```
For each objective:
  - optimize on it
  - add a constraint on it to be always better than the best value found
```

To be used as a subsolver of `LexicoSolver` a solver needs to implement some methods. Its method `implements_lexico_api` should return `True` which means that it implements:
- `get_lexico_objectives_available()`: list of labels available
  corresponding to the internal objectives the solver can optimize.
  Defaults to`problem.get_objective_names()`.
- `set_lexico_objective()`: update the internal objective the subsolver
  will optimize
- `get_lexico_objective_value()`: retrieve the value of the intern
  objective currently optimized
- `add_lexico_constraint()`: add a constraint to the internal model
  on the given objective to avoid worsening it.

_**NB:** the constraints are added on the **internal** objectives of the solver. In some cases, this could be slightly different than the actual objectives computed afterwards with `rcpsp_problem.evaluate()` and stored in the result storage. So it could happen that the actual objectives worsen in a further step._

To illustrate the lexicographic solver, we will use a RCPSP problem. See the [introductory notebook on RCPSP](../RCPSP%20tutorials/RCPSP-1%20Introduction.ipynb) for further details.

## Prerequisites

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization;
- If running on colab, the next cell does it for you;
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

### Imports

In [ ]:
import logging
import random
from typing import Optional

import matplotlib.pyplot as plt
import numpy as np
import yaml

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.generic_tools.callbacks.callback import Callback
from discrete_optimization.generic_tools.callbacks.loggers import ObjectiveLogger
from discrete_optimization.generic_tools.cp_tools import ParametersCp
from discrete_optimization.generic_tools.do_solver import SolverDO
from discrete_optimization.generic_tools.lexico_tools import LexicoSolver
from discrete_optimization.generic_tools.result_storage.result_storage import (
    ResultStorage,
)
from discrete_optimization.rcpsp.parser import get_data_available, parse_file
from discrete_optimization.rcpsp.solvers.cpsat import CpSatCumulativeResourceRcpspSolver

seed = 8851
np.random.seed(seed)
random.seed(seed)

### Logging configuration

We want to display debug logs for discrete-optimization but not for others packages (like matplotlib). We use a logging config defined in a yaml file.

In [ ]:
%%writefile lexico-logging.yml
version: 1
formatters:
  simple:
    format: '%(asctime)s:%(levelname)s:%(message)s'
handlers:
  console:
    class: logging.StreamHandler
    level: DEBUG
    formatter: simple
    stream: ext://sys.stdout
loggers:
  discrete_optimization:
    level: DEBUG
    handlers: [console]
    propagate: False
root:
  level: INFO
  handlers: [console]

In [ ]:
with open("lexico-logging.yml", "rt") as f:
    dico_config = yaml.safe_load(f)

logging.config.dictConfig(dico_config)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j1201_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

In [ ]:
files_available = get_data_available()
file = [f for f in files_available if "j1201_1.sm" in f][0]
rcpsp_problem = parse_file(file)

## Lexico-ready subsolver 

We choose a solver based on [ortools cpsat](https://developers.google.com/optimization/cp/cp_solver) that has the proper methods implemented. 

In [ ]:
subsolver = CpSatCumulativeResourceRcpspSolver(problem=rcpsp_problem)
subsolver.implements_lexico_api()

Le us take a look at available internal objectives:

In [ ]:
subsolver.get_lexico_objectives_available()

## User-defined callbacks

We used the callback API (see corresponding [tutorial](./callbacks.ipynb)) to
- store the internal objectives values at each iteration
- remember which step corresponds to a change of objective

### Store internal objectives

In [ ]:
class InternalObjectivesCallback(Callback):
    def __init__(self, objectives):
        self.objectives = objectives
        self.objectives_values = {obj: [] for obj in objectives}

    def on_step_end(
        self, step: int, res: ResultStorage, solver: SolverDO
    ) -> Optional[bool]:
        # restrict the result storage to last found solution
        res_last = ResultStorage(mode_optim=res.mode_optim, list_solution_fits=res[-1:])
        # get corresponding internal objective value
        for obj in self.objectives:
            self.objectives_values[obj].append(
                solver.get_lexico_objective_value(obj=obj, res=res)
            )

### Store objective changing steps

In [ ]:
class ObjectiveEndStepCallback(Callback):
    def __init__(self, objectives):
        self.objectives = objectives
        self.objective_end_step = {}

    def on_step_end(
        self, step: int, res: ResultStorage, solver: SolverDO
    ) -> Optional[bool]:
        obj = self.objectives[step]
        self.objective_end_step[obj] = len(res)

## Lexico optimization


In [ ]:
solver = LexicoSolver(
    problem=rcpsp_problem,
    subsolver=subsolver,
)
solver.init_model()

# order of objectives
objectives = ["makespan", "used_resource"]

# parameters passed to the subsolver
parameters_cp = ParametersCp.default_cpsat()
time_limit = 4  # timeout for each single-objective optimization
ortools_cpsat_solver_kwargs = dict(random_seed=seed, num_search_workers=1)

# callback to store internal objectives (for future visualization)
internal_obj_cb = InternalObjectivesCallback(objectives)
# callback to store objective changing steps (for future visualization)
obj_end_step_cb = ObjectiveEndStepCallback(objectives)

# lexicographic optimization
result_storage = solver.solve(
    subsolver_callbacks=[ObjectiveLogger(), internal_obj_cb],
    callbacks=[obj_end_step_cb],
    parameters_cp=parameters_cp,
    time_limit=time_limit,
    objectives=objectives,
)

## Visualization

In [ ]:
# colors for each objective
colors = ["tab:blue", "tab:orange"]

# main axe
fig, ax0 = plt.subplots(figsize=(10, 6))
ax0.set_xlabel("Optimization steps")
ax0.set_title(f"Lexicographic optimization: {objectives}")

# first objective
ax = ax0
i_obj = 0
color = colors[i_obj]
obj = objectives[i_obj]
obj_values = internal_obj_cb.objectives_values[obj]
x_steps = list(range(1, len(obj_values) + 1))
ax.set_ylabel(obj, color=color)
ax.plot(x_steps, obj_values, color=color)
ax.tick_params(axis="y", labelcolor=color)

# end of single-objective optimization
ax.axvline(obj_end_step_cb.objective_end_step[obj], color="grey", linestyle="--")

# second objective
ax = ax0.twinx()
i_obj = 1
color = colors[i_obj]
obj = objectives[i_obj]
obj_values = internal_obj_cb.objectives_values[obj]
x_steps = list(range(1, len(obj_values) + 1))
ax.set_ylabel(obj, color=color)
ax.plot(x_steps, obj_values, color=color)
ax.tick_params(axis="y", labelcolor=color)

plt.show()